In [1]:
import numpy as np
import awkward as ak

import hist
from topcoffea.modules.histEFT import HistEFT
import topcoffea.modules.utils as utils

import mplhep as hep
import matplotlib.pyplot as plt

In [2]:
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20, 
          'lines.markersize':5}
plt.rcParams.update(params)

In [3]:
dr_ylabel = r"$d\sigma \: [pb] \; / \; d\Delta R \: [bin]$"
avg_ylabel = r"$d\sigma \: [pb] \; / \; d p_T \: [GeV]$"
l0pt_ylabel = r"$d\sigma \: [pb] \; / \; d p_T \: [GeV]$"

In [4]:
fin1 = "LHCEFT_1410_S1_SM.pkl.gz"
fin2 = "LHCEFT_1410_S2_SM.pkl.gz"
fin3 = "LHCEFT_1410_noEFT_SM.pkl.gz"
hists1 = utils.get_hist_from_pkl(fin1, allow_empty=False)
hists2 = utils.get_hist_from_pkl(fin2, allow_empty=False)
hists3 = utils.get_hist_from_pkl(fin3, allow_empty=False)
print(hists1)

{'avg_top_pt': Hist(Regular(50, 0, 500, name='average top $p_T$ [GeV]'), storage=Weight()) # Sum: WeightedSum(value=12.203, variance=0.0819834) (WeightedSum(value=12.2193, variance=0.0821044) with flow), 'l0pt': Hist(Regular(50, 0, 500, name='leading lepton $p_T$ [GeV]'), storage=Weight()) # Sum: WeightedSum(value=12.2182, variance=0.0821044) (WeightedSum(value=12.2193, variance=0.0821044) with flow), 'dr_leps': Hist(Regular(30, 0, 6, name='$\\Delta R$ (leading lepton, subleading lepton)'), storage=Weight()) # Sum: WeightedSum(value=12.2193, variance=0.0821044), 'mtt': Hist(Regular(80, 0, 2000, name='invariant mass of tops'), storage=Weight()) # Sum: WeightedSum(value=12.2038, variance=0.0820085) (WeightedSum(value=12.2193, variance=0.0821044) with flow), 'njets': Hist(Regular(10, 0, 10, name='njets'), storage=Weight()) # Sum: WeightedSum(value=12.2193, variance=0.0821044)}


In [5]:
def get_ratio_uncertainty(num_hist, denom_hist):
    xvals = num_hist.axes.centers[0]
    yvals_num = num_hist.values()
    yvals_denom = denom_hist.values()
    sigma_num = num_hist.variances()
    sigma_denom = denom_hist.variances()

    ratio = np.divide(yvals_num, yvals_denom)

    # calculation for error propagation for ratio = yavls_num/yvals_denom
    # generally, z=x/y; sigmaz = abs(z)sqrt((sigmax/x)^2+(sigmay/y)^2)
    sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))

    return sigma_y

In [6]:
def get_ratio_points(hist1, histSM):
    num = hist1.values()
    centers = hist1.axes.centers[0]
    denomSM = histSM.values()
    ratio = np.divide(num, denomSM)

    return centers, ratio

In [7]:
h1 = hists1['avg_top_pt']
h2 = hists2['avg_top_pt']
h3 = hists3['avg_top_pt']

centers1, ratio1 = get_ratio_points(h1, h3)
centers2, ratio2 = get_ratio_points(h2, h3)
uncert1 = get_ratio_uncertainty(h1, h3)
uncert2 = get_ratio_uncertainty(h2, h3)

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )

fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(avg_ylabel, fontsize='medium')
ax.set_xlabel("")
plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to SM", fontsize='small')

rax.scatter(centers1, ratio1)
rax.scatter(centers2, ratio2)
rax.errorbar(centers1, ratio1, yerr = None, xerr = uncert1, capsize=5, ls='none')
rax.errorbar(centers2, ratio2, yerr = None, xerr = uncert2, capsize=5, ls='none')

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"average top $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 2])

rax.set_yticks([0,0.5,1,1.5,2])
rax.minorticks_on()
rax.grid(axis='y')

# fig.show()
figname = "redoLHCEFT_SM_avgtoppt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))


plot saved to:  redoLHCEFT_SM_avgtoppt.pdf


In [8]:
h1 = hists1['l0pt']
h2 = hists2['l0pt']
h3 = hists3['l0pt']

centers1, ratio1 = get_ratio_points(h1, h3)
centers2, ratio2 = get_ratio_points(h2, h3)
uncert1 = get_ratio_uncertainty(h1, h3)
uncert2 = get_ratio_uncertainty(h2, h3)

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(l0pt_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.27, 0.83, "Reweighted to SM", fontsize='small')

rax.scatter(centers1, ratio1)
rax.scatter(centers2, ratio2)
rax.errorbar(centers1, ratio1, yerr = None, xerr = uncert1, capsize=5, ls='none')
rax.errorbar(centers2, ratio2, yerr = None, xerr = uncert2, capsize=5, ls='none')

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"leading lepton $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 3])

rax.set_yticks([0,0.5,1,1.5,2, 2.5, 3])
rax.minorticks_on()
rax.grid(axis='y')

# fig.show()
figname = "redoLHCEFT_SM_l0pt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))


plot saved to:  redoLHCEFT_SM_l0pt.pdf


In [9]:
h1 = hists1['dr_leps']
h2 = hists2['dr_leps']
h3 = hists3['dr_leps']

centers1, ratio1 = get_ratio_points(h1, h3)
centers2, ratio2 = get_ratio_points(h2, h3)
uncert1 = get_ratio_uncertainty(h1, h3)
uncert2 = get_ratio_uncertainty(h2, h3)

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")

ax.legend(loc = 'upper right', fontsize = 18)
ax.set_ylabel(dr_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.15, 0.83, "Reweighted to SM", fontsize='small')

rax.scatter(centers1, ratio1)
rax.scatter(centers2, ratio2)
rax.errorbar(centers1, ratio1, yerr = None, xerr = uncert1, capsize=5, ls='none')
rax.errorbar(centers2, ratio2, yerr = None, xerr = uncert2, capsize=5, ls='none')

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"$\Delta$ R (leading lepton, subleading lepton)", fontsize='small')
# rax.set_ylim([0, 3])

rax.set_yticks([0,0.5,1,1.5,2])
rax.grid(axis='y')

# fig.show()
figname = "redoLHCEFT_SM_drleps.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))


plot saved to:  redoLHCEFT_SM_drleps.pdf


In [10]:
fin4 = "LHCEFT_1410_S1_pt1.pkl.gz"
fin5 = "LHCEFT_1410_S2_pt1.pkl.gz"
fin6 = "LHCEFT_1410_noEFT_SM.pkl.gz"
hists4 = utils.get_hist_from_pkl(fin4, allow_empty=False)
hists5 = utils.get_hist_from_pkl(fin5, allow_empty=False)
hists6 = utils.get_hist_from_pkl(fin6, allow_empty=False)

In [11]:
h1 = hists4['avg_top_pt']
h2 = hists5['avg_top_pt']
h3 = hists6['avg_top_pt']

centers1, ratio1 = get_ratio_points(h1, h3)
centers2, ratio2 = get_ratio_points(h2, h3)
uncert1 = get_ratio_uncertainty(h1, h3)
uncert2 = get_ratio_uncertainty(h2, h3)

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )

fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(avg_ylabel, fontsize='medium')
ax.set_xlabel("")
plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to Pt2", fontsize='small')

rax.scatter(centers1, ratio1)
rax.scatter(centers2, ratio2)
rax.errorbar(centers1, ratio1, yerr = None, xerr = uncert1, capsize=5, ls='none')
rax.errorbar(centers2, ratio2, yerr = None, xerr = uncert2, capsize=5, ls='none')

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"average top $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 12])

rax.set_yticks([0,2,4,6,8,10,12])
rax.grid(axis='y')

fig.show()

# fig.show()
figname = "redoLHCEFT_pt2_avgtoppt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))
/tmp/ipykernel_5402/967686744.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


plot saved to:  redoLHCEFT_pt2_avgtoppt.pdf


In [12]:
h1 = hists4['l0pt']
h2 = hists5['l0pt']
h3 = hists6['l0pt']

centers1, ratio1 = get_ratio_points(h1, h3)
centers2, ratio2 = get_ratio_points(h2, h3)
uncert1 = get_ratio_uncertainty(h1, h3)
uncert2 = get_ratio_uncertainty(h2, h3)

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(l0pt_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to Pt2", fontsize='small')

rax.scatter(centers1, ratio1)
rax.scatter(centers2, ratio2)
rax.errorbar(centers1, ratio1, yerr = None, xerr = uncert1, capsize=5, ls='none')
rax.errorbar(centers2, ratio2, yerr = None, xerr = uncert2, capsize=5, ls='none')

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"leading lepton $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 20])

rax.set_yticks([0,5,10,15,20])
rax.grid(axis='y')

# fig.show()
figname = "redoLHCEFT_pt2_l0pt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))


plot saved to:  redoLHCEFT_pt2_l0pt.pdf


In [13]:
h1 = hists4['dr_leps']
h2 = hists5['dr_leps']
h3 = hists6['dr_leps']

centers1, ratio1 = get_ratio_points(h1, h3)
centers2, ratio2 = get_ratio_points(h2, h3)
uncert1 = get_ratio_uncertainty(h1, h3)
uncert2 = get_ratio_uncertainty(h2, h3)

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")

ax.legend(loc = 'upper right', fontsize = 18)
ax.set_ylabel(dr_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to Pt2", fontsize='small')

rax.scatter(centers1, ratio1)
rax.scatter(centers2, ratio2)
rax.errorbar(centers1, ratio1, yerr = None, xerr = uncert1, capsize=5, ls='none')
rax.errorbar(centers2, ratio2, yerr = None, xerr = uncert2, capsize=5, ls='none')

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"$\Delta$ R (leading lepton, subleading lepton)", fontsize='small')
rax.set_ylim([0, 10])

rax.set_yticks([0,2,4,6,8, 10])
rax.grid(axis='y')

# fig.show()
figname = "redoLHCEFT_pt_drleps.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_5402/113102824.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))


plot saved to:  redoLHCEFT_pt_drleps.pdf
